In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
# after running this cell upload the 'makeHistoricalData.py' file

from google.colab import files
files.upload()

Saving makeHistoricalData.py to makeHistoricalData.py


{'makeHistoricalData.py': b'import pandas as pd\nimport numpy as np\n\n# h is the number of days before day (t)\n# r indicates how many days after day (t) --> target-day = day(t+r)\n# target could be number of deaths or number of confirmed \ndef makeHistoricalData(h, r, target, feature_selection):\n        \'\'\' in this code when h is 1, it means there is no history and we have just one column for each covariate\n        so when h is 0, we put h equal to 1, because when h is 0 that means there no history (as when h is 1) \'\'\'\n        if h == 0:\n                h = 1\n        \n        independantOfTimeData = pd.read_csv(\'/content/drive/My Drive/full-fixed-data.csv\')\n        timeDeapandantData = pd.read_csv(\'/content/drive/My Drive/full-temporal-data.csv\')\n\n        allData = pd.merge(independantOfTimeData, timeDeapandantData, on=\'county_fips\')\n        allData = allData.sort_values(by=[\'date\', \'county_fips\'])\n        allData = allData.reset_index(drop=True)\n        #

In [ ]:
# this cell creates two folders to save the results in

!rm -rf Results/
!mkdir Results

!rm -rf Plots/
!mkdir Plots

In [ ]:
import makeHistoricalData

r = 14
target = 'confirmed'
full_fixed_data = pd.read_csv('/content/drive/My Drive/full-fixed-data.csv')
fixed_covariates = list(full_fixed_data.columns.values)

for h in range(1, 8):
    data = makeHistoricalData.makeHistoricalData(h, r, target, 'mrmr')
    # counties = data.county_fips.unique() # if you want to train the model on all counties
    errors = pd.DataFrame()
    counties = [1025, 6033, 13033, 19195, 35013, 36055, 36105]

    for i in range(len(counties)):
        temporal = data.loc[data['county_fips'] == counties[i]] # we construct the (train, val, test) dataframes from temporal dataframe
        temporal.reset_index(drop=True, inplace=True)

        del temporal['date of day t']
        # del temporal['county_name']
        # del temporal['state_name']
        # del temporal['state_fips']
        # del temporal['county_fips']
        temporal = temporal.drop(fixed_covariates, axis = 1)

        totalNumberOfDays = len(temporal)

        # last 14 days are for test dataset and the rest are for train dataset
        train = temporal.head(totalNumberOfDays-28)
        validation = temporal.loc[totalNumberOfDays-28:totalNumberOfDays-15, :]
        test = temporal.tail(14)

        X_train = train.drop('Target', axis=1)
        y_train = train['Target']

        ####################################### deleting first days that have zero value for target
        index_of_first_nonZero_day = y_train.ne(0).idxmax()
        index_of_first_nonZero_day = index_of_first_nonZero_day - 7
        if index_of_first_nonZero_day > 0 and counties not in [36043, 36055, 36059]:
            X_train = X_train.iloc[index_of_first_nonZero_day:]
            y_train = y_train.iloc[index_of_first_nonZero_day:]
        #######################################

        X_val = validation.drop('Target', axis=1)
        y_val = validation['Target']

        X_test = test.drop('Target', axis=1)
        y_test = test['Target']
        
        numberOfCovariates = X_train.shape[1]

        for c in range(numberOfCovariates):
            print("***********************************************************")
            print("Working on county = "+str(counties[i])+", h = "+str(h)+", c = "+str(c+1))
            print("***********************************************************")
            train_X = X_train.iloc[:, 0:c+1].copy()
            train_y = y_train.copy()

            val_X = X_val.iloc[:, 0:c+1].copy()
            val_y = y_val.copy()

            test_X = X_test.iloc[:, 0:c+1].copy()
            test_y = y_test.copy()

            ######################################### Normalizing
            scalar = preprocessing.MinMaxScaler()

            train_X = train_X.values
            train_X = scalar.fit_transform(train_X)

            val_X = val_X.values
            val_X = scalar.fit_transform(val_X)

            test_X = test_X.values
            test_X = scalar.fit_transform(test_X)

            train_y = train_y.values
            train_y = scalar.fit_transform(train_y.reshape(-1, 1))

            val_y = val_y.values
            val_y = scalar.fit_transform(val_y.reshape(-1, 1))

            # test_y = test_y.values
            # test_y = scalar.fit_transform(test_y.reshape(-1, 1))

            train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
            val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
            test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

            train_y = train_y.reshape((train_y.shape[0]), )
            val_y = val_y.reshape((val_y.shape[0]), )
            # test_y = test_y.reshape((test_y.shape[0]), )
            #########################################

            print(train_X.shape, test_X.shape)
            print(train_y.shape, test_y.shape)

            ############################################# defining the model
            model = Sequential()
            model.add(LSTM(8, return_sequences=True, input_shape=(1, train_X.shape[2])))
            model.add(LSTM(256, dropout=0.2, return_sequences=True))
            model.add(LSTM(256, dropout=0.2, return_sequences=True))
            model.add(LSTM(128, dropout=0.2, return_sequences=True))
            model.add(LSTM(128))
            model.add(Dense(1, activation='selu'))

            # opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
            model.compile (
                loss='mean_squared_error',
                optimizer=keras.optimizers.Adam(0.01)
            )
            ############################################# end of defining the model

            history = model.fit (
                train_X, train_y,
                epochs=50,
                batch_size=128,
                validation_data=(val_X, val_y),
                verbose=1,
                shuffle=False
            )

            # fig = plt.figure()
            # plt.plot(history.history['loss'], label='train')
            # plt.plot(history.history['val_loss'], label='test')
            # plt.xlabel('Epoch')
            # plt.ylabel('Loss')
            # plt.legend()
            # plt.show()
            # fig_name = 'h=' + str(h) + ', county_fips=' + str(counties[i])
            # fig.savefig(fig_name, dpi=600, quality=100)

            # getting the result on test dataset
            pred = model.predict(test_X)
            pred = scalar.inverse_transform(pred.reshape(-1, 1))
            # y_test = scalar.inverse_transform(y_test.reshape(-1, 1))

            test_y = test_y.to_numpy()
            test_y = test_y.reshape(-1, 1)
            pred = pred.reshape(-1, 1)
            pred = np.round(pred)

            ##################################### storing predictinos in a file
            result = pd.DataFrame()
            result['Prediction'] = pred.tolist()
            result['Actual'] = y_test.tolist()
            file_name = '/content/Results/predictions_'+str(counties[i])+'_'+str(h)+'_'+str(c+1)+'.csv'
            result.to_csv(file_name, mode='w', index=False)
            #####################################

            ##################################### ploting y_test and pred in a same graph
            days = [x for x in range(1,15)]
            fig = plt.figure()
            plt.plot(days, test_y.tolist(), label='Actual')
            plt.plot(days, pred.tolist(), label='Prediction')
            plt.xlabel("Day")
            plt.ylabel("Target")
            plt.legend()
            fig_name = '/content/Plots/'+str(counties[i])+'_'+str(h)+'_'+str(c+1)+'.png'
            fig.savefig(fig_name, dpi=600)
            #####################################

            pred[pred < 0] = 0
            pred[pred == 0] += 1
            test_y = np.round(test_y)
            test_y[test_y == 0] = 1

            MAE = mean_absolute_error(test_y, pred)
            MSE = (np.square(test_y - pred)).mean(axis=None)
            RMSE = np.sqrt(MSE)        
            MAPE = np.mean(np.abs((test_y - pred) / test_y)) * 100
            r2 = r2_score(test_y, pred)

            key = str(counties[i])+'_'+str(h)+'_'+str(c+1)
            temp_list = [[key, MAE, MSE, RMSE, MAPE, r2]]
            errors = errors.append(pd.DataFrame(temp_list, columns=['county and h and c', 'MAE', 'MSE','RMSE', 'MAPE', 'adj_r_squared']), ignore_index=True)

        file_name = '/content/Results/errors_per_county_h='+str(h)+'_c='+str(c+1)+'.csv'
        errors.to_csv(file_name, mode='w', index=False)

***********************************************************
Working on county = 1025, h = 1, c = 1
***********************************************************
(60, 1, 1) (14, 1, 1)
(60,) (14,)
Train on 60 samples, validate on 14 samples
Epoch 1/50
60/60 [==============================] - 3s 56ms/step - loss: 0.0982 - val_loss: 0.1769
Epoch 2/50
60/60 [==============================] - 0s 393us/step - loss: 0.0790 - val_loss: 0.1338
Epoch 3/50
60/60 [==============================] - 0s 347us/step - loss: 0.0612 - val_loss: 0.0918
Epoch 4/50
60/60 [==============================] - 0s 349us/step - loss: 0.0671 - val_loss: 0.0968
Epoch 5/50
60/60 [==============================] - 0s 334us/step - loss: 0.0614 - val_loss: 0.1049
Epoch 6/50
60/60 [==============================] - 0s 395us/step - loss: 0.0579 - val_loss: 0.1104
Epoch 7/50
60/60 [==============================] - 0s 397us/step - loss: 0.0574 - val_loss: 0.1135
Epoch 8/50
60/60 [==============================] - 0s 332us/ste

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


***********************************************************
Working on county = 13033, h = 1, c = 2
***********************************************************
(69, 1, 2) (14, 1, 2)
(69,) (14,)
Train on 69 samples, validate on 14 samples
Epoch 1/50
69/69 [==============================] - 2s 23ms/step - loss: 0.0850 - val_loss: 0.3224
Epoch 2/50
69/69 [==============================] - 0s 338us/step - loss: 0.0712 - val_loss: 0.2805
Epoch 3/50
69/69 [==============================] - 0s 321us/step - loss: 0.0607 - val_loss: 0.2382
Epoch 4/50
69/69 [==============================] - 0s 293us/step - loss: 0.0708 - val_loss: 0.2491
Epoch 5/50
69/69 [==============================] - 0s 295us/step - loss: 0.0634 - val_loss: 0.2606
Epoch 6/50
69/69 [==============================] - 0s 316us/step - loss: 0.0605 - val_loss: 0.2678
Epoch 7/50
69/69 [==============================] - 0s 398us/step - loss: 0.0601 - val_loss: 0.2720
Epoch 8/50
69/69 [==============================] - 0s 313us/st

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


***********************************************************
Working on county = 13033, h = 1, c = 3
***********************************************************
(69, 1, 3) (14, 1, 3)
(69,) (14,)
Train on 69 samples, validate on 14 samples
Epoch 1/50
69/69 [==============================] - 2s 29ms/step - loss: 0.0850 - val_loss: 0.3233
Epoch 2/50
69/69 [==============================] - 0s 497us/step - loss: 0.0715 - val_loss: 0.2826
Epoch 3/50
69/69 [==============================] - 0s 349us/step - loss: 0.0610 - val_loss: 0.2398
Epoch 4/50
69/69 [==============================] - 0s 311us/step - loss: 0.0690 - val_loss: 0.2491
Epoch 5/50
69/69 [==============================] - 0s 329us/step - loss: 0.0635 - val_loss: 0.2604
Epoch 6/50
69/69 [==============================] - 0s 379us/step - loss: 0.0606 - val_loss: 0.2679
Epoch 7/50
69/69 [==============================] - 0s 308us/step - loss: 0.0600 - val_loss: 0.2724
Epoch 8/50
69/69 [==============================] - 0s 409us/st

...

***********************************************************
Working on county = 19195, h = 2, c = 17
***********************************************************
(53, 1, 17) (14, 1, 17)
(53,) (14,)
Train on 53 samples, validate on 14 samples
Epoch 1/50
53/53 [==============================] - 1s 27ms/step - loss: 0.0566 - val_loss: 0.0033
Epoch 2/50
53/53 [==============================] - 0s 460us/step - loss: 0.0534 - val_loss: 0.0083
Epoch 3/50
53/53 [==============================] - 0s 365us/step - loss: 0.0546 - val_loss: 0.0071
Epoch 4/50
53/53 [==============================] - 0s 384us/step - loss: 0.0542 - val_loss: 0.0037
Epoch 5/50
53/53 [==============================] - 0s 370us/step - loss: 0.0534 - val_loss: 0.0013
Epoch 6/50
53/53 [==============================] - 0s 423us/step - loss: 0.0538 - val_loss: 0.0016
Epoch 7/50
53/53 [==============================] - 0s 353us/step - loss: 0.0537 - val_loss: 0.0033
Epoch 8/50
53/53 [==============================] - 0s 356us

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


***********************************************************
Working on county = 19195, h = 2, c = 18
***********************************************************
(53, 1, 18) (14, 1, 18)
(53,) (14,)
Train on 53 samples, validate on 14 samples
Epoch 1/50
53/53 [==============================] - 2s 31ms/step - loss: 0.0566 - val_loss: 0.0027
Epoch 2/50
53/53 [==============================] - 0s 479us/step - loss: 0.0534 - val_loss: 0.0097
Epoch 3/50
53/53 [==============================] - 0s 444us/step - loss: 0.0551 - val_loss: 0.0077
Epoch 4/50
53/53 [==============================] - 0s 554us/step - loss: 0.0543 - val_loss: 0.0042
Epoch 5/50
53/53 [==============================] - 0s 376us/step - loss: 0.0535 - val_loss: 0.0019
Epoch 6/50
53/53 [==============================] - 0s 381us/step - loss: 0.0536 - val_loss: 0.0015
Epoch 7/50
53/53 [==============================] - 0s 362us/step - loss: 0.0537 - val_loss: 0.0025
Epoch 8/50
53/53 [==============================] - 0s 394us

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


***********************************************************
Working on county = 19195, h = 2, c = 19
***********************************************************
(53, 1, 19) (14, 1, 19)
(53,) (14,)
Train on 53 samples, validate on 14 samples
Epoch 1/50


In [ ]:
# run this cell to make zip file from results and then download them :)

!zip -r /content/Results.zip /content/Results/
!zip -r /content/Plots.zip /content/Plots/